In [39]:
import numpy as np
import keras
import os
import tensorflow as tf
import pandas as pd

from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3

from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential,load_model
from keras import backend as K

import matplotlib.pyplot as plt

from PIL import Image

In [40]:
def preprocess(img):
    """
    Reshape an image to 224 x 224 x 3, so that it can be understood by our model
    
    Args:
        image: an arbitary height x width x 3 image
    Output:
        returns the converted image
    """
    
    width, height = img.shape[0], img.shape[1]
    
    img = image.array_to_img(img, scale=False)
    
    desired_width, desired_height = 224, 224
    
    if width < desired_width:
        desired_width = width
    if height < desired_height:
        desired_height = height
    
    start_x = np.maximum(0, int((width-desired_width)//2))
    
    img = img.crop((start_x, np.maximum(0, height-desired_height), start_x + desired_width, height))
    
    img = img.resize((224, 224))
    
    img = image.img_to_array(img)
    
    return img / 255  
  

In [41]:
def get_images(path, correct_images, is_train=True):
    """
    Get all the images
    
    Args:
        path               : folder path to load images from 
        correct_images     : image names which have protest label set
        is_train           : to identify if the images to load are training images or test images
    Output:
        returns an array of processed images 
    """
    
    files = os.listdir(path)
    
    files = list(map(lambda x: os.path.join(path, x), files)) # load all the images
    
    if is_train:
        filtered_files = [image for image in files if image in correct_images]
    else:
        filtered_files = [image for image in files if image in correct_images]
    
    
    train_images = filtered_files
    
    images = []
    for item in train_images:
        img = image.load_img(item)
        img = image.img_to_array(img)
        img = preprocess(img)
        images.append(img)
    
    return images

In [42]:
def convert_labels_to_numpy_array(items): 
    """
    Filter all labels based on the protest parameter
    
    Args:
        items: a dataframe object of csv file
    Output:
        returns an array of numpy arrays    
    """
    values = []
    
    for index, row in items.iterrows():

        cats = {
                'protest': True,
                'violence': True,
                'sign': True,
                'photo': True,
                'fire': True,
                'police': True,
                'children': True,
                'group_20': True,
                'group_100': True,
                'flag': True,
                'night': True,
                'shouting': True
               }

        if row['protest']:
            
            array = []
            
            for cat in cats:
                f = float(row[cat])
                array.append(f)
            
            narray = np.array(array)
        
            values.append(narray)

    return values

In [43]:
def get_labels(path):
    """
    Load the csv file and return an array of numpy arrays of it's objects
    
    Args:
        path: a relative path to load csv file from
    Output:
        returns an array of the numpy objects
    """
    labels = pd.read_csv(path)
    
    train_labels = convert_labels_to_numpy_array(labels)  
    
    return train_labels

In [44]:
def reshape_images(images, size):
    """
    Reshape the images array adding one more dimension as batch size
    
    Args:
        images : list of images
        size   : batch size
    Output:
        returns the numpy array of reshaped objects
    """
    return np.asarray(images).reshape(size,224,224,3)

In [45]:
def get_model(base_model):
    """
    Get the base model and modify it 
    
    Args:
        base_model: It is the object of our pretrained model
    Output:
        returns the new_model by adding new layers and setting first few layers as 
        non trainable
    """
    bottleneck_input = base_model.get_layer(index=0).input
    bottleneck_output = base_model.get_layer(index=-2).output
    
    bottleneck_model = Model(inputs=bottleneck_input, outputs=bottleneck_output)
    
    for layer in bottleneck_model.layers:
        
        layer.trainable = False
    
    new_model = Sequential()
    
    new_model.add(bottleneck_model)
    
    new_model.add(Dense(12))
    
    new_model.add(Dropout(0.5))
    
    new_model.add(Dense(12, input_dim=2048, activation='softmax'))
    
    return new_model

In [46]:
# initialize our base model
base_model = InceptionV3()

# get the new modified model
new_model = get_model(base_model)

# compile the model with optimizer, loss function and add a metrics for accuracy
new_model.compile(optimizer='rmsprop', loss='mean_absolute_error', metrics=['accuracy'])

In [ ]:
# training labels file path
training_labels_path = "training.csv"

# create a dataframe object by loading that file using pandas
df = pd.read_csv(training_labels_path)

# store the image names which have protest label set
train_image_names = {}

# iterate over the file and modify the file name to "type/filename". Here type is either train or test.
# this will help us in further mapping labels and images with each other
for index, item in df.iterrows():
    
    if item['protest']:
        name = "train/" + item['fname']
        train_image_names[name] = True
        
# test labels file path
test_labels_path = "test.csv"

# read the test csv file
test_df = pd.read_csv(test_labels_path)

# store the test image names which have protest label set
test_image_names = {}

# iterate over the file and modify the file name to "type/filename". Here type is either train or test.
# this will help us in further mapping labels and images with each other
for index, item in test_df.iterrows():
    
    if item['protest']:
        name = "test/" + item['fname']
        
        test_image_names[name] = True

# path of training images folder
training_images_path = "train/"

# path of test images folder
test_images_path = "test/"

# get all the training labels
train_labels = get_labels(training_labels_path)

# get all the training images
train_images = get_images(training_images_path, train_image_names, True)

# get all the test labels
test_labels = get_labels(test_labels_path)

# get all the test images
test_images = get_images(test_images_path, test_image_names, False)

# size of our training images dataset
train_size = len(train_images)

# size of our test images dataset
test_size = len(test_images)

# validation size of data
validation_size = test_size // 3

# reshaped train images
reshaped_train_images = reshape_images(train_images, train_size)

# reshaped test images
reshaped_test_images = reshape_images(test_images[validation_size:test_size], test_size - validation_size)


# validation images
validation_images = reshape_images(test_images[0:validation_size], validation_size)



# validation labels 
validation_labels = test_labels[0:validation_size]



# test labels 
new_test_labels = test_labels[validation_size:test_size]

In [ ]:
# initialize our base model
base_model = InceptionV3()

# get the new modified model
new_model = get_model(base_model)

# compile the model with optimizer, loss function and add a metrics for accuracy
new_model.compile(optimizer='rmsprop', loss='mean_absolute_error', metrics=['accuracy'])
# fit our training images dataset
new_model.fit(reshaped_train_images, np.array(train_labels),epochs=1, shuffle=True)

# save the state of our model, so that we won't need to run it again from scratch in future
new_model.save('news_image_with_dropout_with_dropout_twelve.h5')

Epoch 1/1
 768/9316 [=>............................] - ETA: 19:05 - loss: 0.2683 - acc: 0.1862

In [ ]:
# predict label of our test images
new_model = load_model('news_image_with_dropout_with_dropout_twelve.h5')
predict = new_model.predict(reshaped_test_images, verbose=1)


In [ ]:
# calculate accuracy of our model
new_model.evaluate(test_images_dataset, np.array(test_labels_dataset), verbose=0)

In [ ]:
test_labels = test_labels[781:]
predictY = predict
testY = np.array(test_labels)
predicted_violence = []
predicted_protest = []
predicted_sign = []
actual_protest = []
actual_violence = []
actual_sign = []
actual_photo = []
predicted_photo = []
actual_fire = []
predicted_fire = []
for i in range(0,5,1):
    predicted_protest.append(predictY[i][0])
    predicted_violence.append(predictY[i][1])
    predicted_sign.append(predictY[i][2])
    predicted_photo.append(predictY[i][3])
    predicted_fire.append(predictY[i][4])
    #print(predicted)
    actual_protest.append(testY[i][0])
    actual_violence.append(testY[i][1])
    actual_sign.append(testY[i][2])
    actual_photo.append(testY[i][3])
    actual_fire.append(testY[i][4])
    #print(actual)

    
#plt.figure(figsize=(10,4))

fig,axs = plt.subplots(5, 1)

axs[0].plot(actual_protest, 'r-', label='Actual')
axs[0].plot(predicted_protest, 'g-', label='Predicted')
axs[0].set_title('Protest')
fig.suptitle('Categorical Predicted Values', fontsize=16)


axs[1].plot(actual_violence, 'r-', label='Actual')
axs[1].plot(predicted_violence, 'g-', label='Predicted')
axs[1].set_title('Violence')

axs[2].plot(actual_sign, 'r-', label='Actual')
axs[2].plot(predicted_sign, 'g-', label='Predicted')
axs[2].set_title('Sign')

axs[3].plot(actual_photo, 'r-', label='Actual')
axs[3].plot(predicted_photo, 'g-', label='Predicted')
axs[3].set_title('Photo')

axs[4].plot(actual_fire, 'r-', label='Actual')
axs[4].plot(predicted_fire, 'g-', label='Predicted')
axs[4].set_title('Fire')

plt.show()